In [1]:
# ... existing code ...
import time
from flask import Flask, request, jsonify, Response
import json
from tensor_utils_pybind import  tensor_restore_from_handler_pybind, IPCHandleManager

app = Flask(__name__)

@app.route('/merged_handler', methods=['POST'])
def merged_handler():
    DEBUG = False
    start_time = time.time()

    hidden_states_meta = json.loads(request.form['hidden_states_meta']) 
    if DEBUG:
        print(f"hidden_states_meta: {hidden_states_meta}")
    
    topk_weights_meta = json.loads(request.form['topk_weights_meta']) 
    if DEBUG:
        print(f"topk_weights_meta: {topk_weights_meta}")
    
    topk_ids_meta = json.loads(request.form['topk_ids_meta']) 
    if DEBUG:
        print(f"topk_ids_meta: {topk_ids_meta}")

    handler = request.files['handler'].read()
    handle_manager = IPCHandleManager(handler, hidden_states_meta['device'])


    hidden_states = tensor_restore_from_handler_pybind(handle_manager, hidden_states_meta)
    topk_weights = tensor_restore_from_handler_pybind(handle_manager, topk_weights_meta)
    topk_ids = tensor_restore_from_handler_pybind(handle_manager, topk_ids_meta)
    end_time = time.time()
    
    handle_manager.close_ipc_handle()
    
    print(f"hidden_states: {hidden_states}")
    print(f"topk_weights: {topk_weights}")
    print(f"topk_ids: {topk_ids}")
    print(f"restored 3 tensors in : {(end_time - start_time)*1000} ms")
    
    response = {
        'message':"ok",
    }
    
    return jsonify(response)

@app.route('/merged_single', methods=['POST'])
def test():
    merged_handler = request.files['merged_handler'].read()
    

    hidden_states_meta = json.loads(request.form['hidden_states_meta']) 

    
    print(f"hidden_states_meta: {hidden_states_meta}")

    
    handle_manager = IPCHandleManager(merged_handler, hidden_states_meta['device'])
    
        # 使用同一个handle管理器创建所有tensor
    global hidden_states
    hidden_states = tensor_restore_from_handler_pybind(handle_manager, hidden_states_meta,True)
  
    print(f"merged: {hidden_states}")


    response = {
        'message':"ok",
        # 'restored_tensor':t.cpu().tolist(),  # Convert to list for JSON serialization
    }
    
    return jsonify(response)

if __name__ == '__main__':
    app.run(port=1177)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:1177
Press CTRL+C to quit
Exception ignored in: <function IPCHandleManager.__del__ at 0x73108ce71d80>
Traceback (most recent call last):
  File "/root/vllm/ipc_handler_demo/exp_field/tensor_utils_pybind.py", line 58, in __del__
    self.close_ipc_handle()
  File "/root/vllm/ipc_handler_demo/exp_field/tensor_utils_pybind.py", line 55, in close_ipc_handle
    ipc_tensor_pybind.close_ipc_handle(self.dev_ptr)
RuntimeError: cudaIpcCloseMemHandle failed: invalid argument
127.0.0.1 - - [04/Jun/2025 10:12:46] "POST /merged_handler HTTP/1.1" 200 -


offset_ptr: 0x73106c200000
tensor.data_ptr: 0x73106cc00000
offset_ptr: 0x73106ca00000
tensor.data_ptr: 0x731070800000
offset_ptr: 0x73106ca02000
tensor.data_ptr: 0x731070802000
hidden_states: tensor([[-0.4055, -0.5077,  1.3535,  ..., -0.4322, -0.6301, -0.6791],
        [-0.2482, -0.2057, -0.9148,  ..., -0.4003, -1.0719,  0.2630],
        [ 1.6116,  1.6385,  0.9094,  ...,  0.5347, -0.6238, -0.7083],
        ...,
        [ 0.5343, -0.9832,  0.9822,  ...,  0.7232, -0.1213, -1.6530],
        [-0.1056,  0.0625,  0.4504,  ...,  0.2294, -2.3053,  1.1465],
        [ 0.0213, -0.1226, -2.0355,  ...,  0.0794,  1.3142,  0.7398]],
       device='cuda:1')
topk_weights: tensor([[ 0.0153,  0.2168,  0.3789,  0.7266],
        [ 0.2334, -1.7578,  1.6328,  0.0918],
        [-0.1523, -0.1279,  0.9219, -0.4004],
        ...,
        [-1.4062, -0.1504,  0.1631, -0.4902],
        [-0.8867, -0.3945, -0.1816, -1.1797],
        [ 0.5547,  1.2031,  0.9336,  0.1338]], device='cuda:1',
       dtype=torch.bfloat16)
